# 04 — Re-Ranking

`$vectorSearch` is fast but approximate. A **cross-encoder re-ranker** (`rerank-2.5`) jointly processes each *(query, document)* pair and produces a precise relevance score — far more accurate, but only practical on a small candidate set.

The standard two-stage pipeline:
```
All docs  →  $vectorSearch (top-50 ANN)  →  rerank-2.5  →  top-5 final results
```

**Steps:**
1. `$vectorSearch` to retrieve candidates
2. Re-rank with `rerank-2.5` — observe the rank shifts
3. Full pipeline: `$vectorSearch` → rerank → LLM-ready context

In [ ]:
// ── Setup ────────────────────────────────────────────────────────────────────
import { MongoClient } from 'mongodb';

// ← Paste your VoyageAI API key here (get one at https://dash.voyageai.com)
const VOYAGE_API_KEY = 'pa-...';
const QUERY_MODEL    = 'voyage-4-lite';
const RERANK_MODEL   = 'rerank-2.5';
const INDEX_NAME     = 'listing_vector_index';  // created in notebook 01

const client = new MongoClient(process.env.MONGODB_URI!);
await client.connect();
const db  = client.db('voyage_lab');
const col = db.collection<{ _id: string; [key: string]: unknown }>('listings');

console.log('Connected. Docs with embeddings:', await col.countDocuments({ embedding: { $exists: true } }));

In [ ]:
// ── Helpers ───────────────────────────────────────────────────────────────────
async function embed(texts: string[], inputType: 'document' | 'query'): Promise<number[][]> {
  const res = await fetch('https://api.voyageai.com/v1/embeddings', {
    method: 'POST',
    headers: { 'Content-Type': 'application/json', 'Authorization': `Bearer ${VOYAGE_API_KEY}` },
    body: JSON.stringify({ input: texts, model: QUERY_MODEL, input_type: inputType }),
  });
  if (!res.ok) throw new Error(await res.text());
  const json = await res.json() as { data: { embedding: number[] }[] };
  return json.data.map(d => d.embedding);
}

async function rerank(
  query: string,
  documents: string[],
  topK?: number,
): Promise<{ index: number; relevance_score: number; document: string }[]> {
  const res = await fetch('https://api.voyageai.com/v1/rerank', {
    method: 'POST',
    headers: { 'Content-Type': 'application/json', 'Authorization': `Bearer ${VOYAGE_API_KEY}` },
    body: JSON.stringify({ query, documents, model: RERANK_MODEL, top_k: topK, return_documents: true }),
  });
  if (!res.ok) throw new Error(await res.text());
  const json = await res.json() as { data: { index: number; relevance_score: number; document: string }[] };
  return json.data;
}

console.log('Helpers defined.');

## Step 1 — `$vectorSearch` to retrieve candidates

In [ ]:
// ── Retrieve top-20 candidates with $vectorSearch ─────────────────────────────
const QUERY   = 'family-friendly house with garden and outdoor space for kids';
const [qVec]  = await embed([QUERY], 'query');

const candidates = await col.aggregate([
  {
    $vectorSearch: {
      index:         INDEX_NAME,
      path:          'embedding',
      queryVector:   qVec,
      numCandidates: 100,
      limit:         20,
    },
  },
  {
    $project: {
      name:          1,
      description:   1,
      property_type: 1,
      price:         1,
      embedScore:    { $meta: 'vectorSearchScore' },
    },
  },
]).toArray();

console.log('Top 5 by $vectorSearch:');
candidates.slice(0, 5).forEach((c, i) =>
  console.log(`  ${i+1}. [${(c.embedScore as number).toFixed(4)}] ${c.name}`)
);

## Step 2 — Re-rank candidates with `rerank-2.5`

The re-ranker reads the **full text** of each document alongside the query — far richer than embedding similarity.

In [ ]:
// ── Re-rank ───────────────────────────────────────────────────────────────────
const t0 = Date.now();
const rerankResults = await rerank(QUERY, candidates.map(c => String(c.description ?? c.name)), 5);
console.log(`Re-rank latency: ${Date.now() - t0}ms\n`);

console.log('Top 5 AFTER re-ranking (with original $vectorSearch rank):');
rerankResults.forEach((r, newRank) => {
  const doc = candidates[r.index];
  console.log(`  ${newRank+1}. [rerank=${r.relevance_score.toFixed(4)}, was #${r.index+1}] ${doc.name}`);
});
// OBSERVE: Some documents move up significantly — the cross-encoder catches nuances the embedding missed.

In [ ]:
// ── Rank shift table ──────────────────────────────────────────────────────────
const top10 = await rerank(QUERY, candidates.map(c => String(c.description ?? c.name)), 10);

const comparison = top10.map((r, newRank) => ({
  name:        String(candidates[r.index].name).substring(0, 42).padEnd(42),
  embedRank:   r.index + 1,
  rerankPos:   newRank + 1,
  shift:       (r.index + 1) - (newRank + 1),   // positive = moved up
  rerankScore: r.relevance_score.toFixed(4),
}));

console.table(comparison);
// Large positive shift = embedding undervalued this doc.
// Large negative shift = embedding overvalued it.

## Step 3 — Full RAG pipeline

`$vectorSearch` → `rerank-2.5` → formatted context string ready for an LLM.

In [ ]:
// ── RAG: retrieve + rerank → context string ───────────────────────────────────
async function retrieveContext(userQuery: string, topK = 3): Promise<string> {
  // 1. Embed the query
  const [qv] = await embed([userQuery], 'query');

  // 2. $vectorSearch — fetch 20 candidates from MongoDB
  const cands = await col.aggregate([
    {
      $vectorSearch: {
        index:         INDEX_NAME,
        path:          'embedding',
        queryVector:   qv,
        numCandidates: 100,
        limit:         20,
      },
    },
    { $project: { name: 1, description: 1, price: 1, property_type: 1 } },
  ]).toArray();

  // 3. Re-rank the candidates
  const ranked = await rerank(userQuery, cands.map(c => String(c.description ?? c.name)), topK);

  // 4. Build context string from re-ranked results
  return ranked.map((r, i) => {
    const d = cands[r.index];
    return `[${i+1}] ${d.name} ($${d.price}/night, ${d.property_type})\n${d.description}`;
  }).join('\n\n');
}

const context = await retrieveContext('quiet retreat for remote work with fast wifi');
console.log('Retrieved context (ready for LLM):\n');
console.log(context);

In [ ]:
// ── Cleanup ───────────────────────────────────────────────────────────────────
await client.close();
console.log('Done.');